<a href="https://colab.research.google.com/github/sourav7sharma/Subtitle-Canvas/blob/main/Subtitle_Canvas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instructions to generate Character Canvas from SRT File**

---


1. As you open this website, you will see a connect button at the top right of the screen. Use this button everytime to connect to the remote server.
2. Once the connection is completed it will show RAM and Disk status bars. Click on the folder icon on the extreme left menu, this is the current folder you will be working in.
3. Now, right click on the empty space below the sample_data folder, then upload the srt file there.
4. Once the file is uploaded, right click on it and rename it to srt_file.srt.
5. Once that is done, simply run the below code cell using the play button that will appear on its top left when you put your cursor in that cell.
6. After, the run is complete which will take about 2-3 minutes, a new file will be generate named "character.json. Download this file and use it accordingly.

In [ ]:
!pip install pysrt pydantic openai==0.28

import pysrt
import openai
from pydantic import BaseModel, Field
from typing import List
from datetime import timedelta
import json
from time import sleep
from google.colab import userdata


import base64
en_api_key ="Your api"

CHARACTER_PROMPT = '''
You are an expert computer artist that uses an AI text to image tool to generate picture perfect portrait images.
The AI requires accurate description for the character with immense facial details, hairstyle details, age group, gender and body structure.
If the story itself does not have much context to fill all the details, you as an artist must completely imagine the character from head to toe and provide the description.
You will create expert detailed text prompts for different characters of the story that will be used by the image generation AI. In the prompt, only put keywords that are relevant to imagine the character's looks separated by commas. Always mention the gender in the prompt as well.
Examples of the output are,
First Name - Quinn
Gender - Male
Body details - A teenage male, oval shaped face with pointed chin and pimples on his face, big brown coloured eyes, lean muscular build

First Name - Jake
Gender - Male
Body details - A teenage male, square shaped face, light stubble, thin build, dark coloured short hair, wavy hair

First Name - Emma
Gender - Female
Body details - A teenage female, beautiful face, thin lips, medium length blonde hair, big bright eyes, thin eyebrows

You have to follow the exact approach for every character in the story
'''

class ChatHistory:
    def __init__(self) -> None:
        self.chat_messages = []

    def add_message(self, role, message):
        self.chat_messages.append({
            "role": role,
            "content": message
        })

        if len(self.chat_messages) >= 9:
            self.chat_messages = self.chat_messages[:1] + self.chat_messages[-6:]

    def get_message_history(self):
        return self.chat_messages

    def reset(self):
        self.chat_messages = []

    def reset_with_system(self):
        self.chat_messages = self.chat_messages[:1]

class CharacterImage(BaseModel):
    first_name: str = Field(..., description="First Name of the character")
    gender: str = Field(..., description="Gender of the character")
    body_details: str = Field(..., description="description of the character's body details")

class CharactersList(BaseModel):
    characters: List[CharacterImage] = Field(..., description="list of all the characters in the story")

def get_script_from_srt(srt_filename):
  subs = pysrt.open(srt_filename)
  script = ""
  for sub in subs:
    script += sub.text

  return script

class OpenAIService:
    def gpt3prompt2(self, messages, functions, model="gpt-4o"):
        trials = 10
        arguments = None
        completion = None
        while trials:
            try:
                completion = openai.ChatCompletion.create(
                    model=model,
                    messages=messages,
                    functions=functions,
                    function_call="auto"
                )
                output_txt = completion.get('choices')[0].message["function_call"]["arguments"]
                arguments = json.loads(output_txt)
                break
                # print(completion)
            except Exception as e:
                trials -= 1
                print(completion)
                sleep(5)
                print(e)
                continue

        return {"characters": []} if arguments is None else arguments

    def get_characters_prompt(self, script: str, messages: ChatHistory):
        prompt = f'''
Provide prompts for character from the following script:
{script}
        '''
        messages.add_message(role="user", message=prompt)
        functions = [
                        {
                            "name": "get_characters_prompts",
                            "description": "Get character prompts with their name, gender and prompt",
                            "parameters": CharactersList.schema()
                        }
                    ]
        characters = self.gpt3prompt2(messages.get_message_history(), functions=functions)
        messages.add_message(role="assistant", message="")
        return characters

def get_characters(script):
    batches = []
    batch = ""
    for sent in script.split("."):
      sent += "."
      batch += sent
      if len(batch) > 5000:
        batches.append(batch)
        batch = ""

    if batch:
      batches.append(batch)

    characters_library = {}
    service = OpenAIService()
    character_messages = ChatHistory()
    character_messages.add_message(role="system", message=CHARACTER_PROMPT)

    for batch in batches:
      characters = service.get_characters_prompt(batch, character_messages)
      characters = characters["characters"]


      for character in characters:
          characters_library[character['first_name'].split(" ")[0]] = {
              'body_details': character['body_details'],
              'gender': character['gender']
          }

    with open("character.json", "w", encoding="utf-8") as file:
        json.dump(characters_library, file, indent=2)

srt_filepath = "/content/srt_file.srt"
get_characters(get_script_from_srt(srt_filepath))







new section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')